# Deform a hemisphere into a cow's head

---


This notebook attempts to solve the problem of deforming a hemisphere into a cow's head.


Import the required modules.


In [ ]:
import numpy as np
import pyvista as pv
import pycpd as cpd

---

## 1 - Create source and target meshes


Create a mesh of a hemisphere using the `pv.Sphere()` function.


In [ ]:
hemisphere_mesh = pv.Sphere(
    radius=0.5, theta_resolution=100, phi_resolution=100, end_phi=90
)
hemisphere_mesh.plot(show_edges=True)

Import a mesh of a cow's head from PyVista. A cow's head is a complicated geometry which is topologically equivalent to the hemisphere mesh. It should be a good mesh to benchmark different deformation algorithms.


In [ ]:
cow_head_filename = pv.examples.download_cow_head(load=False)
cow_head_mesh = pv.get_reader(cow_head_filename).read()
cow_head_mesh.plot(show_edges=True)

Rotate and translate the cow's head mesh so that it lines up nicely with the hemisphere mesh.


In [ ]:
cow_head_mesh = cow_head_mesh.rotate_x(angle=90)
cow_head_mesh = cow_head_mesh.rotate_y(angle=-62)
cow_head_mesh = cow_head_mesh.translate((-0.8, 0, -3.7))

Plot the source and target mesh.


In [ ]:
plotter = pv.Plotter()
plotter.add_mesh(hemisphere_mesh, color="lightblue")
plotter.add_mesh(cow_head_mesh, color="brown", opacity=0.5, show_edges=True)
plotter.show()

---

## 2 - Use `pycpd` to deform the hemisphere to the cow's head


First, we will try using the `pypcd` [1] package to deform the hemisphere mesh to the cow's head mesh. `pypcd` is a NumPy implementation of the Coherent Point Drift (CPD) algorithm by Myronenko and Song [2].

After running 5 iterations, the general shape of the cow's head is captured fairly well. However, the ears and horns of the cow are not distinct, and the hole for the neck is in completely the wrong place!


In [ ]:
NUM_ITERATIONS = 5

hemisphere_mesh_deformed = hemisphere_mesh.copy()

for i in range(NUM_ITERATIONS):
    # Create a deformable registration object.
    deformable_registration = cpd.DeformableRegistration(
        alpha=0.05,
        X=np.array(cow_head_mesh.points),
        Y=np.array(hemisphere_mesh_deformed.points),
    )

    # Run the deformable registration, and update the hemisphere mesh.
    transformed_hemisphere_points, _ = deformable_registration.register()
    hemisphere_mesh_deformed.points = transformed_hemisphere_points

# Plot the meshes in separate subplots.
pl = pv.Plotter(shape=(1, 2))

pl.subplot(0, 0)
pl.add_mesh(hemisphere_mesh_deformed, color="lightblue")
pl.add_text("Deformed hemisphere", font_size=10)

pl.subplot(0, 1)
pl.add_mesh(cow_head_mesh, color="orange")
pl.add_text("Cow's head", font_size=10)

pl.show()

In [ ]:
pl = pv.Plotter()
pl.add_mesh(hemisphere_mesh_deformed, color="lightblue")
pl.show()

---

## 3 - Use a neural network to deform the hemisphere to the cow's head


---

## References


[1] - Khallaghi, S. siavashk/pycpd. (2025) (https://github.com/siavashk/pycpd).

[2] - Myronenko, A. & Song, X. Point-Set Registration: Coherent Point Drift. IEEE Trans. Pattern Anal. Mach. Intell. 32, 2262–2275 (2010) (https://doi.org/10.1109/TPAMI.2010.46).
